# Import Google Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#!unzip "/content/drive/MyDrive/paris.zip" -d "/content/drive/MyDrive/Paris120x120/"  > /dev/null

# Carica DataSet

In [ ]:
import load_dataset
user="Daniele"

load_dataset.set_user(user)
groundtruth_dir, dataset_dir = load_dataset.get_dirs()

In [ ]:
class_names = load_dataset.get_classnames()
num_classes = len(class_names)

dataset = load_dataset.get_dataset(groundtruth_dir, dataset_dir, class_names)

Funzioni per la stampa di statistiche sul DB

In [ ]:
import print_functions as prt

prt.print_dataset_stats(dataset)

In [ ]:
#prt.print_class_good_images(dataset,0) #stampa tutte le immagini good della classe 0

# Split dei dati in training, validation, test set

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import random

Dataset Shuffle

In [ ]:
#load_dataset.shuffle(dataset)

Dataset Split

In [ ]:
# Da Dataset -> A X_train, X_val, X_test

# per avere uno split equilibrato, esso avverrà secondo il seguente criterio:
# il training set sarà formato, per ogni landmark, dall' 80% del totale delle
# good per quel landmark + 80% del totale delle ok per quel landmark + ecc...
# il test set e il validation set avranno il 10% a testa

def split_dataset(dataset, x_train, x_val, x_test):

  for i in range(len(dataset)):
    for j in range(len(dataset[i])): 
      
      num_samples = len(dataset[i][j])

      train_len = round(0.8 * num_samples)
      x_train[i][j] = x_train[i][j] + dataset[i][j][:train_len]
      
      val_len = (num_samples - train_len) // 2
      x_val[i][j] = x_val[i][j] + dataset[i][j][train_len: train_len + val_len]

      test_len = (num_samples - train_len - val_len)
      x_test[i][j] = x_test[i][j] + dataset[i][j][-test_len:]

  return;

x_train = [ [[],[],[],[]] for x in range(len(class_names))]
x_val = [ [[],[],[],[]] for x in range(len(class_names))]
x_test = [ [[],[],[],[]] for x in range(len(class_names))]

split_dataset(dataset, x_train, x_val, x_test)

x_train_len = prt.dataset_len(x_train)
x_val_len = prt.dataset_len(x_val)

print(prt.dataset_len(x_train))
print(prt.dataset_len(x_val))
print(prt.dataset_len(x_test))

# Data Augmentation

Definizione degli ImageDataGenerator. Gli oggetti ImageDataGenerator definiscono le trasformazioni che si faranno.
Chiamando flow(numpy_array_immagini) su ImageDataGenerator, flow restituirà un generatore di immagini transformate.

In [ ]:
#oggetto ImageDataGenerator, definisce le trasformazioni
gen_settings = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=60,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  
    vertical_flip=False,                     
    rescale=None,
    fill_mode='nearest'
)

#ImageDataGenerator senza trasformazioni per validation set
test_val_settings = keras.preprocessing.image.ImageDataGenerator()

Ora definiamo una classe custom che prende gli ImageDataGenerator del nostro dataset, e gli usa per creare un generatore di triplette.

In [ ]:
class CustomDataGenerator(object):
    
    # parametro nel costruttore: dataset_generators. Ha la solita struttura [ [[good],[ok],[bad],[junk]],  [...], ...]
    # però invece che avere le foto nei good, ok, ... ho un generatore per quella categoria che genera le trasformazioni
    
    def __init__(self, dataset_generators,batch_size) -> None:
        self.dataset_generators = dataset_generators
        self.batch_size=batch_size

    def next_train(self):
        current_class = -1
        while 1:
            anchors=[]
            positives=[]
            negatives=[]
            for i in range(self.batch_size):
                current_class = current_class+1
                current_class = current_class % num_classes
                
                _anchor,_positive,_negative = retrieve_single_triplet(self,current_class)
                anchors.append(_anchor)
                positives.append(_positive)
                negatives.append(_negative)
            
            anchors = tf.convert_to_tensor(anchors)
            positives = tf.convert_to_tensor(positives)
            negatives = tf.convert_to_tensor(negatives)

            triplet = (anchors, positives, negatives)
            yield(triplet)
        
def retrieve_single_triplet(self,current_class):
    different_class = random.randint(0, num_classes-1)
    
    while(different_class == current_class):
        different_class = random.randint(0, num_classes-1)  
            
    _anchor = next(self.dataset_generators[current_class][1])[0]
    _positive = next(self.dataset_generators[current_class][1])[0]
    _negative = next(self.dataset_generators[different_class][random.choice([1,3])])[0]    
    return _anchor,_positive,_negative

x_train, x_val, x_test sono strutture dati di immagini così disposte: [ [[good],[ok],[bad],[junk]],  [...], ...]
Sostituiamo le immagini coi generatori creati a partire da quelle immagini.

In [ ]:
import numpy as np

#x_train, x_val, x_test prima contenevano array di immagini. Ora contengono generatori separati per classe e qualità
def transform_dataset(db, gen_settings):
    ds = [ [[],[],[],[]] for x in range(len(class_names))]
    for i in range(len(ds)):
        for j in [1,3]: #abbiamo solo immagini ok e junk (1 e 3) nel database
            generatore = gen_settings.flow(x=np.asarray(db[i][j]), batch_size=1) #batch size 1 per ritornare una sola img
            ds[i][j] = generatore
    return ds

x_train_gen = transform_dataset(x_train, gen_settings)
x_val_gen = transform_dataset(x_val, test_val_settings)
#x_test = transform_dataset(x_test, test_val_settings)

In [ ]:
batch_size=128
#Training generator
custom_gen_train = CustomDataGenerator(x_train_gen,batch_size)
generatore_triplete_train = custom_gen_train.next_train()

#Validation generator
custom_gen_val = CustomDataGenerator(x_val_gen,batch_size)
generatore_triplete_val = custom_gen_val.next_train() 

In [ ]:
triplete_batch = next(generatore_triplete_train)

ancore = triplete_batch[0]
positivi = triplete_batch[1]
negativi = triplete_batch[2]

tripla1 = (ancore[0], positivi[0], negativi[0])
tripla2 = (ancore[1], positivi[1], negativi[1])
tripla3 = (ancore[2], positivi[2], negativi[2])

prt.stampa_tripletta(tripla1)
prt.stampa_tripletta(tripla2)
prt.stampa_tripletta(tripla3)

# Definizione Modello Triple

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet
from tensorflow.keras.applications.resnet import preprocess_input
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7

In [ ]:
def get_encoder (input_shape):
    
    
    #base_cnn = resnet.ResNet50(
    #base_cnn = resnet.ResNet101(
    #base_cnn = resnet.ResNet152(
    base_cnn = EfficientNetB7(
        weights="imagenet", input_shape=input_shape, include_top=False)
    
    trainable = False
    for layer in base_cnn.layers:
        if layer.name == "conv5_block1_out":
            trainable = True
        layer.trainable = trainable
        
    encode_model = tf.keras.models.Sequential([
        base_cnn,
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(32)
    ], name="Embedding")
    
    
    return encode_model

In [ ]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

In [ ]:
def get_siamese_network (input_shape = (120,120,3)):
    embedding_model = get_encoder(input_shape)
    input_anchor = tf.keras.layers.Input(shape= input_shape)
    input_positive = tf.keras.layers.Input(shape= input_shape)
    input_negative = tf.keras.layers.Input(shape= input_shape)
    
    #embedding_anchor = embedding_model(resnet.preprocess_input(input_anchor))
    #embedding_positive = embedding_model(resnet.preprocess_input(input_positive))
    #embedding_negative = embedding_model(resnet.preprocess_input(input_negative))

    embedding_anchor = embedding_model(input_anchor)
    embedding_positive = embedding_model(input_positive)
    embedding_negative = embedding_model(input_negative)

    distances = DistanceLayer()(
    embedding_anchor ,
    embedding_positive,
    embedding_negative ,
    )

    siamese_network =  tf.keras.models.Model(
    inputs=[input_anchor, input_positive, input_negative], outputs=distances
    )
    return siamese_network
    
siamese_network = get_siamese_network()  

In [ ]:
class SiameseModel(tf.keras.models.Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super().__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.acc_tracker =tf.keras.metrics.BinaryAccuracy(name ="accuracy")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        
        with tf.GradientTape() as tape:
            loss,acc = self._compute_loss(data)
        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )
        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state((acc|True),acc)
        return {"loss": self.loss_tracker.result(),"acc": self.acc_tracker.result()}

    def test_step(self, data):
        
        loss,acc = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state((acc|True),acc)
        return {"loss": self.loss_tracker.result(),"acc": self.acc_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss, (ap_distance<an_distance)
    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker,self.acc_tracker]

In [ ]:
lr=0.00100
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(lr), weighted_metrics=[])

In [ ]:
PATIENCE_END=10
PATIENCE_REDUCE=3
COOLDOWN=3
VERBOSE=1
NAME=f"EfficientNetB7_{lr}"
EPOCHS=100

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=PATIENCE_END,
                                                    mode='min',
                                                    restore_best_weights='val_acc')

reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.5,
                                               patience=PATIENCE_REDUCE,
                                               verbose=VERBOSE,
                                               mode='auto',
                                               min_delta=0.01,
                                               cooldown=COOLDOWN,
                                               min_lr=1e-6
                                              )
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/{NAME}")

# siccome uso un generator, la dimensione del dataset diventa potenzialmente infinita
# devo definire a quanti batch corrisponde un'epoca -> steps per epoch, validation steps
steps_train=int( x_train_len/batch_size )
steps_val=int(x_val_len/batch_size)

#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=6, restore_best_weights='val_acc')
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, restore_best_weights='val_acc')

history = siamese_model.fit(
    generatore_triplete_train,
    steps_per_epoch=steps_train,
    batch_size=batch_size,
    epochs=EPOCHS,
    verbose=1,
    validation_data=generatore_triplete_val,
    validation_steps=steps_val,
    callbacks=[early_stopping,reduce_lr,tensorboard_callback]
)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy & loss')
plt.ylabel('accuracy & loss')
plt.xlabel('epoch')
plt.legend(['acc train', 'acc test', 'loss train','loss test'], loc='upper left')
plt.show()

# Usare il Modello Allenato

Runnare questa cella solo se non si ha il modello "encoder" salvato

In [ ]:
def get_encoder_with_weights(model):
    encoder=get_encoder((120,120,3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight= e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = get_encoder_with_weights(siamese_model)
#encoder.summary()

Runnare questa cella solo se si sono già calcolati i pesi per il modello "encoder"

In [ ]:
#encoder=get_encoder((120,120,3))
#encoder.load_weights("PesiMigliori")

Funzioni per cambiare la rappresentazione da [[ [good], [ok], [bad], [junk] ],[...], ...] ad un vettore normale + vettore di label y.

In [ ]:
#prende tutte le immagini good e ok da data e le mette in un array arr, con le immagini ordinate per classe crescente
def to_array(data):
    arr=[]
    for index in range(num_classes):
        arr.extend(data[index][0]) #prendo le immagini good della classe index, e le appendo in arr
        arr.extend(data[index][1]) #prendo le immagini ok della classe index, e le appendo in arr
    print("Data transformed into array")
    return arr

#array che segna dove finiscono in arr le immagini della classe i-esima
def get_levels(data):
    y_levels=[]
    previous = 0
    for index in range(num_classes):
        i_class_len = len(data[index][0])+len(data[index][1])
        y_levels.append(previous+i_class_len)
        previous = previous + i_class_len
    return y_levels

#ritorna il vettore y di label(le classi) a partire da un y_level
def get_y(y_levels):
    tot = y_levels[-1]
    y = [np.zeros(num_classes) for i in range(tot)]
    class_index = 0
    
    for i in range(tot):
        while i >= y_levels[class_index]:
            class_index = class_index + 1
        y[i][class_index] = 1
    return y

def produce_encodings(x, encoder):
    dt = tf.data.Dataset.from_tensor_slices(x)
    dt = dt.batch(1,drop_remainder=False)
    print("Encoding data...")
    return encoder.predict(dt)

Passo le immagini di training attraverso l'encoder e produco un encoding delle immagini di training

In [ ]:
y_train_levels = get_levels(x_train)
x_train_array = to_array(x_train)
x_train_encoded = produce_encodings(x_train_array, encoder)

Passo le immagini di test attraverso l'encoder e produco un encoding delle immagini di test

In [ ]:
y_test_levels = get_levels(x_test)
x_test_array = to_array(x_test)
x_test_encoded = produce_encodings(x_test_array, encoder)

Per ogni immagine di test, ottengo un vettore con le probabilità che appartenga ad una certa classe (come il layer softmax)

In [ ]:
def get_predizioni(x_train_encoded, x_test_encoded, y_train_levels):
    predizioni = []
    for i in range(len(x_test_encoded)):
        distances = tf.reduce_sum(tf.square(x_train_encoded - x_test_encoded[i]), -1)

        x = []  
        for index in range(num_classes):
            if index==0:
                diClasse=distances[0:y_train_levels[index]]
            else:
                diClasse=distances[y_train_levels[index-1]:y_train_levels[index]]
            x.append( tf.math.reduce_min(diClasse) )
        distribution = tf.nn.softmax(x)
        distribution = tf.negative(distribution)

        predizioni.append(distribution.numpy().tolist())
    return predizioni

predizioni = get_predizioni(x_train_encoded, x_test_encoded, y_train_levels)

Calcolo e stampa del topk

In [ ]:
from statistics import mean

def topk(num):
    x = tf.keras.metrics.TopKCategoricalAccuracy(k=num)
    x.update_state(get_y(y_test_levels),predizioni)
    return x.result().numpy()

k = [0,0,0,0] #top k=1,k=2,k=3,k=4

for i in range(len(k)):
    k[i] = topk(i+1)

print("Valori del topk:", k)
print("Valore medio del topk:", mean(k))

(FACOLTATIVO/NON SERVE)

Printiamo una matrice artigianale di predizioni sul test set (ci dà un idea se siamo sulla buona strada). Una matrice diagonale è l'ideale (darebbe un top k=1 uguale a 1.0). Attenzione che se il modello predice le classi con la stessa identica probabilità, nella matrice metto come predizione la prima classe nell'ordine dei class names

In [ ]:
matrice = [[0 for i in range(len(predizioni[0]))] for i in range(num_classes)]

o = 0
for i in range(len(predizioni)):
    if y_test_levels[o]-1 < i:
        o = o+1
    index = 0
    m = predizioni[i][0]
    for j in range(len(predizioni[i])):
        if predizioni[i][j] > m:
            m = predizioni[i][j]
            index = j
    matrice[o][index] = matrice[o][index] + 1
        
print("Le righe sono il ground truth. Cioè riga i = classe i");
print("Le colonne sono le predizioni.\nUn buon lavoro restituisce una matrice con la diagonale piena e il resto vuoto");

for i in range(num_classes):
    print(matrice[i])

# Salvare i Pesi

In [ ]:
old_encoder = get_encoder((120,120,3))
old_encoder.load_weights("PesiMigliori")

x_train_encoded = produce_encodings(x_train_array, old_encoder)
x_test_encoded = produce_encodings(x_test_array, old_encoder)

predizioni = get_predizioni(x_train_encoded, x_test_encoded, y_train_levels)
old_k = [0,0,0,0] #top k=1,k=2,k=3,k=4

for i in range(len(old_k)):
    old_k[i] = topk(i+1)

print("Valori del topk:", k)
print("Valore medio del topk:", mean(k))

print("Valori del topk vecchio:", old_k)
print("Valore medio del topk vecchio:", mean(old_k))

if mean(k) > mean(old_k):
    print("Nuovi pesi salvati")
    encoder.save_weights("PesiMigliori")
else:
    print("Nuovi pesi NON salvati") 

In [ ]:
encoder.save_weights("Pesi_Dell_Ultima_Run")